# Indexing Chat Text

In [1]:
import os
from typing import List

import redis
import pandas as pd
from langchain.document_loaders import TextLoader, CSVLoader
from langchain.vectorstores.redis import Redis
from langchain.schema import Document
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
CACHE_TYPE = os.getenv("CACHE_TYPE")
REDIS_URL = os.getenv("REDIS_URL")
OPENAI_COMPLETIONS_ENGINE = os.getenv("OPENAI_COMPLETIONS_ENGINE")
INDEX_NAME = os.getenv("INDEX_NAME")

## 1. Loading Chat Text

The [starter notebook](./0.0.1-sample-LLM-Stack-Hackathon-Starter.ipynb) used manual indexing with schema that's not compatible with Langchain internals.

Thus, we are encoding the embeddings and indexing everything again.

In [3]:
r = redis.StrictRedis.from_url(REDIS_URL)
r.flushall()

True

In [4]:
datasource = pd.read_csv(
    '../data/chats.csv'
).to_dict("records")

chat_texts = [
    Document(
        page_content=doc["chat_text"][:3000],
        metadata={
            "channel_name": doc["channel_name"],
            "thread_id": doc["thread_id"],
        }
    ) for doc in datasource
]

## 2. Indexing

In [5]:
embeddings = OpenAIEmbeddings()
vectorstore = Redis.from_documents(
    documents=chat_texts,
    embedding=embeddings,
    index_name='chat_index',
    redis_url=REDIS_URL,
)

In [6]:
vectorstore.similarity_search("foobar")

[Document(page_content='U019ABXBYET: <https://bard.google.com/>', metadata={'channel_name': 'generative-ai', 'thread_id': '2023-03-24 15:56:52.108999 UTC'}),
 Document(page_content='U01G8F6E38T: <https://www.youtube.com/watch?v=OWIxzE2D7Xk>', metadata={'channel_name': 'random', 'thread_id': '2023-04-24 17:24:07.087339 UTC'}),
 Document(page_content='U0221S0CLJU: <https://nayya.slack.com/archives/C03DS0Y129H/p1675186024430969|https://nayya.slack.com/archives/C03DS0Y129H/p1675186024430969>', metadata={'channel_name': 'mlops-alerts', 'thread_id': '2023-01-31 17:19:40.964499 UTC'}),
 Document(page_content='U041C7A71UG: o0+UP3T8K9M5: happens to the best of us', metadata={'channel_name': 'learning-resources', 'thread_id': '2022-09-22 02:59:29.472529 UTC'})]